In [ ]:
import pandas as pd
import numpy as np

path = "../d/Global_Mobility_Report.csv.xz"
df = pd.read_csv(path, parse_dates=["date"])

# Filter to national 
national_row = lambda df : df[df.sub_region_1.isna() & df.sub_region_2.isna() & df.metro_area.isna()]
nat = national_row(df)
nat = nat.drop("parks_percent_change_from_baseline", axis=1)
nat = nat.drop(["country_region_code", "sub_region_1", "sub_region_2", "metro_area", "iso_3166_2_code", "census_fips_code", "place_id"], axis=1)

subs = {"Czechia": 'Czech Republic'}
nat["country_region"] = nat["country_region"].replace(subs)
nat.rename(columns={"date":"Date", "country_region": "Region Name"}, inplace=True)
nat["Date"] = pd.to_datetime(nat["Date"], utc=True)
nat.set_index(["Region Name", "Date"])
nat

In [ ]:
dfb = pd.read_csv("../d/modelBrauner_dataBraunerTE.csv", parse_dates=["Date"])
dfb = dfb.merge(nat, on=["Region Name", "Date"], how="left")
dfb.fillna(0.0, inplace=True)
MOBS = {
    "retail_and_recreation_percent_change_from_baseline": "Recre",
    "grocery_and_pharmacy_percent_change_from_baseline": "Groc",
    "transit_stations_percent_change_from_baseline": "Trans",
    "workplaces_percent_change_from_baseline": "Work",
    "residential_percent_change_from_baseline": "Resid",
    }
mobility = np.mean([np.float32(dfb[m]) for m in MOBS], axis=0)
mobility
dfb.drop(MOBS, inplace=True, axis=1)
dfb["Mobility decrease"] = mobility / -100.0
dfb.to_csv(f"../d/modelBrauner_dataBraunerTE_mobility_{''.join(MOBS.values())}.csv", index=False)